In [72]:
import random
from tqdm import tqdm
from math import exp
from math import log1p
import numpy as np
import pandas as pd

One instance

In [106]:
class Being():
    
    weights = []
    variables = []
    e = []
    eW = []
    ln = []
    lnW = []
    features = 0
    pruning = 1
    weight_sigma = 20
    variable_power = 3
    lr = 3
    
    def __init__(self, features, variable_power=3, weight_sigma=20, lr = 3, pruning=1, parents=[]):
        if parents == []:
            self.features = features
            self.variable_power = variable_power
            self.weight_sigma = weight_sigma
            self.lr = lr
            self.pruning = pruning
            self.variables = []
            self.weights = []
            self.e = []
            self.ln = []
            self.eW = []
            self.lnW = []
            self.setup()
        else:
            self.features = parents[0].features
            self.variable_power = parents[0].variable_power
            self.weight_sigma = parents[0].weight_sigma
            self.lr = lr
            self.pruning = parents[0].pruning
            self.variables = []
            self.weights = []
            self.e = []
            self.ln = []
            self.eW = []
            self.lnW = []
            self.mutate(parents)
    
    def setup(self):
        for i in range(int(self.features*self.variable_power*random.random()*2+1)):
            temp = [random.randint(0, self.features-1)]
            for i in range(int(self.variable_power*random.random())):
                temp.append(random.randint(0, self.features-1))
            self.variables.append(temp)
        for i in range(len(self.variables)-1):
            if random.random() > .95:
                if random.random() > .5:
                    self.e.append(self.variables[i])
                else:
                    self.ln.append(self.variables[i])
        for i in range(len(self.variables)-1):
            self.weights.append(random.gauss(0, self.weight_sigma))
        for i in range(len(self.e)-1):
            self.eW.append(random.gauss(0, self.weight_sigma))
        for i in range(len(self.ln)-1):
            self.lnW.append(random.gauss(0, self.weight_sigma))

    
    def mutate(self, parents):
        for parent in parents:
            for var, weight in zip(parent.variables, parent.weights):
                if random.random() < self.pruning/len(parents):
                    self.variables.append(var)
                    self.weights.append(weight+random.gauss(0, self.lr))
            for e, eW in zip(parent.e, parent.eW):
                if random.random() < self.pruning/len(parents):
                    self.e.append(e)
                    self.eW.append(eW+random.gauss(0, self.lr))
            for ln, lnW in zip(parent.ln, parent.lnW):
                if random.random() < self.pruning/len(parents):
                    self.ln.append(ln)
                    self.lnW.append(lnW+random.gauss(0, self.lr))
        for i in range(int(self.pruning/random.random())):
            temp = [random.randint(0, self.features-1)]
            for i in range(int(self.variable_power*random.random())):
                temp.append(random.randint(0, self.features-1))
            if random.random() > .99:
                if random.random() > .5:
                    self.e.append(temp)
                    self.eW.append(random.gauss(0, self.weight_sigma))
                else:
                    self.ln.append(temp)
                    self.lnW.append(random.gauss(0, self.weight_sigma))
            self.variables.append(temp)
            self.weights.append(random.gauss(0, self.weight_sigma)) 
            
    
    def predict(self, x):
        pred = []
        for data in x:
            result = 0.0
            for var, weight in zip(self.variables, self.weights):
                mult = weight
                for elem in var:
                    mult *= data[elem]
                result += mult
            for e, eW in zip(self.e, self.eW):
                mult = 1
                for elem in e:
                    mult *= data[elem]
                try:
                    result += eW*exp(mult)
                except:
                    result += eW*exp(1/mult)
            for ln, lnW in zip(self.ln, self.lnW):
                mult = 1
                for elem in ln:
                    mult *= data[elem]
                result += lnW*log1p(abs(mult))
            pred.append(result)
        return pred
    
    def evaluate(self, x, y):
        pred = self.predict(x)
        correct = 0
        incorrect = 0
        for y1, yp1 in zip(y, pred):
            if (yp1 >= 0 and y1 >= .5) or (yp1 < 0 and y1 < .5):
                correct += 1
            else:
                incorrect += 1
        return (correct+0.0)/(correct+incorrect+0.0)                
        

In [107]:
a = Being(3)
a.predict(np.array([[1, 1, 1]]))

[-54.271597896715285]

Creating and Killing Instances

In [108]:
class Slaughterhouse():
    
    population = 0
    random = .3
    top = 20
    being = 0
    decay = 1
    lr = 0.001
    beings = []
    parents = []
    outlast = 0.01
    
    def __init__(self, being, population, random=0.4, top=20, decay=1, outlast=0.01):
        self.population = population
        self.random = random
        self.top = top
        self.being = being
        self.decay = decay
        self.lr = being.lr
        self.beings = []
        self.parents = []
        self.outlast = outlast
    
    def create_generation(self):
        for i in range(self.population):
            self.beings.append([0, Being(self.being.features, variable_power=self.being.variable_power, weight_sigma=self.being.weight_sigma, lr=self.being.lr, pruning=self.being.pruning)])
    
    def score_generation(self, x, y):
        for i in tqdm(range(len(self.beings))):
            self.beings[i][0] = self.beings[i][1].evaluate(x, y)
        self.beings = sorted(self.beings, key=lambda x: x[0], reverse=True)
    
    def next_generation(self):
        self.get_parents()
        self.beings = []
        for i in range(self.population):
            if random.random() > self.random:
                self.beings.append([0, Being(self.being.features, parents=self.parents, lr=self.lr/self.decay)])
            else:
                if random.random() > .5:
                    self.beings.append([0, Being(self.being.features, parents=[self.parents[int(random.random()*len(self.parents))]], lr=self.lr/self.decay)])
                else:
                    self.beings.append([0, Being(self.being.features, variable_power=self.being.variable_power, weight_sigma=self.being.weight_sigma, lr=self.being.lr, pruning=self.being.pruning)])
    
    def get_parents(self):
        self.parents = []
        high = self.beings[0][0]
        for i in range(self.top):
            if high - self.outlast < self.beings[i][0]:
                self.parents.append(self.beings[i][1])
                print('Parent Acc:', self.beings[i][0])
    
    def go(self, x, y, generations=1, init=True):
        if init:
            print('Start Generation')
            self.create_generation()
            self.score_generation(x, y)
        for i in range(generations):
            print('Generation ', i+1)
            self.next_generation()
            self.score_generation(x, y)
        self.get_parents()
        return self.parents
        

Using Titanic Data

In [109]:
titanic = pd.read_csv("titanic_dropped.csv")
x = np.array(titanic.drop(['Survived'], axis=1))
y = np.array(titanic['Survived'])

In [110]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)

In [111]:
template = Being(7, lr=5, variable_power=6, weight_sigma=20)

In [112]:
house = Slaughterhouse(template, 1000, decay=.95)

In [113]:
final = house.go(x_train, y_train, generations=3)

Start Generation


  0%|          | 0/1000 [00:00<?, ?it/s]

Generation  1
Parent Acc: 0.7950963222416813
Parent Acc: 0.7915936952714536
Parent Acc: 0.7898423817863398
Parent Acc: 0.7898423817863398
Parent Acc: 0.7880910683012259
Parent Acc: 0.7880910683012259


100%|██████████| 1000/1000 [00:26<00:00, 37.15it/s]


Generation  2
Parent Acc: 0.8108581436077058
Parent Acc: 0.809106830122592
Parent Acc: 0.8073555166374781
Parent Acc: 0.8073555166374781
Parent Acc: 0.8056042031523643
Parent Acc: 0.8056042031523643
Parent Acc: 0.8038528896672504
Parent Acc: 0.8038528896672504
Parent Acc: 0.8038528896672504
Parent Acc: 0.8021015761821366
Parent Acc: 0.8021015761821366
Parent Acc: 0.8021015761821366
Parent Acc: 0.8021015761821366


100%|██████████| 1000/1000 [00:28<00:00, 35.20it/s]


Generation  3
Parent Acc: 0.8248686514886164
Parent Acc: 0.8178633975481612
Parent Acc: 0.8178633975481612
Parent Acc: 0.8161120840630472


100%|██████████| 1000/1000 [00:41<00:00, 23.92it/s]

Parent Acc: 0.8336252189141856
Parent Acc: 0.830122591943958
Parent Acc: 0.8283712784588442
Parent Acc: 0.8283712784588442
Parent Acc: 0.8283712784588442
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8266199649737302
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164
Parent Acc: 0.8248686514886164


In [114]:
final[0].predict(x_test)

[-676253.12540654768,
 72726.930084550622,
 -2399556.0242570997,
 4331049.8398739416,
 -4756093.2152004475,
 539612.11792611098,
 -21239624.890262194,
 -1857719.5429636659,
 -4208133.7337791398,
 180130.52665807321,
 37469804.664364792,
 -670511.81435965875,
 1781482.5243680736,
 876895.58371750696,
 -6014089.4008604996,
 -9282059.4291629307,
 720716.41058997414,
 -39493011.789943188,
 158546.79666656395,
 19041401.473637164,
 -812999.53180563077,
 -118417356.60317244,
 -5395263.8028721875,
 -1686363.122214776,
 5863135.9418260511,
 -4441275.0692513138,
 -1734970.4256925683,
 -11363135.936041759,
 -3381096.0836841534,
 -7485382.5957333324,
 1466201.1065470269,
 44138149.358638123,
 -5394822.1299760649,
 -1103510.2261467976,
 -4750851.2323633637,
 17281510.416846167,
 -114454340.90344688,
 -1726478.199454275,
 -17737273.744152222,
 5582985.4080503639,
 -16331591.946434518,
 155237.81931461761,
 -551711.95682673715,
 -17787723.87786651,
 -3688791.1534283981,
 171767.11392704665,
 229139.

In [115]:
y_pred = final[0].predict(x_test)
correct = 0
incorrect = 0
for y1, yp1 in zip(y_test, y_pred):
    if (yp1 >= 0 and y1 >= .5) or (yp1 < 0 and y1 < .5):
        correct += 1
    else:
        incorrect += 1
(correct+0.0)/(correct+incorrect+0.0)

0.7762237762237763